In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_absolute_error,mean_squared_error

%matplotlib inline

sns.set_style('darkgrid')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

# Getting data

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
testfin=pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
subm=pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
testfin

In [ ]:
train

# Analysis

In [ ]:
train.columns[train.dtypes=='object']

In [ ]:
for i in train.columns[train.dtypes=='object']:
    print(train[i].value_counts())

In [ ]:
oh=ColumnTransformer([('encoder',
                       OneHotEncoder(sparse=False,drop='first'),
                       ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8','cat9'],
                      )],
                    remainder='passthrough')

In [ ]:
train=train.drop(['id'],axis=1)
X=train.drop(['target'],axis=1)
y=train['target']

In [ ]:
X

# Train Test Split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [ ]:
x_train

# Encoding (One Hot Encoding)

In [ ]:
x_train=oh.fit_transform(x_train)
x_val=oh.transform(x_val)
x_test=oh.transform(x_test)

In [ ]:
x_train

In [ ]:
x_train[0]

In [ ]:
testfin=testfin.drop(['id'],axis=1)
test_fin=oh.transform(testfin)

# **Linear Regression**

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)

pred1=lr.predict(x_test)

print(f"RMSE :{mean_squared_error(y_test,pred1)}")
# print(F"mae:{mean_absolute_error(y_test,pred1)}")

plt.figure(figsize=(10,10))

plt.plot(pred1,y_test)



# **SGD Regression**

In [ ]:
lr1=SGDRegressor(verbose=1)
lr1.fit(x_train,y_train)

pred2=lr1.predict(x_test)

print(f"RMSE :{mean_squared_error(y_test,pred2)}")

# print(F"MAE:{mean_absolute_error(y_test,pred2)}")

plt.figure(figsize=(10,10))

plt.plot(pred2,y_test)


# **Tensorflow Model**

In [ ]:
model=Sequential()

model.add(Dense(x_train.shape[1],activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(2*x_train.shape[1],activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2*x_train.shape[1],activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(x_train.shape[1],activation='relu'))

model.add(Dense(1,activation='relu'))

model.compile(optimizer='RMSprop',loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
e1=EarlyStopping(verbose=1)

In [ ]:
history=model.fit(x=x_train,y=y_train,epochs=50,callbacks=[e1],workers=5,validation_data=(x_val,y_val))
pred3=model.predict(x_test)

In [ ]:
d1=pd.DataFrame(history.history)
# d1.iloc[:,[1,3]]
d1.iloc[:,[1,3]].plot(figsize=(8,8))

In [ ]:
print(f"RMSE :{mean_squared_error(y_test,pred3,squared=False)}")
model.summary()

# **Output**

In [ ]:
prediction=lr.predict(test_fin)
subm['target']=prediction
subm.to_csv(f'lr1.csv',index=False)